In [1]:
pip install cupy-cuda11x

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd 
import ast
import numpy as np
import pickle
import cupy

In [8]:
# Import the Popularity datset to recommend the user
# We recommend them by the movie which they havnet seen, movies seen by the similar users 
# and order will be according to the popularity of the movie
Popularity_data = pd.read_csv("popular_movies_by_id.csv.")
movie_order = Popularity_data['movieId'].values

In [10]:
Popularity_data
movie_order.shape

(45115,)

In [14]:
ratings = pd.read_csv("data/ratings.csv")
ratings.drop("timestamp",axis = 1,inplace=True)
ratings

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0
...,...,...,...
26024284,270896,58559,5.0
26024285,270896,60069,5.0
26024286,270896,63082,4.5
26024287,270896,64957,4.5


In [18]:
movies_info =  pd.read_csv("data/movies_metadata.csv",low_memory=False)
movies_info.head()


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [25]:
user_rating_data = ratings
user_rating_data


,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0
...,...,...,...
26024284,270896,58559,5.0
26024285,270896,60069,5.0
26024286,270896,63082,4.5
26024287,270896,64957,4.5


In [34]:
user_rating_data.drop_duplicates()

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0
...,...,...,...
26024284,270896,58559,5.0
26024285,270896,60069,5.0
26024286,270896,63082,4.5
26024287,270896,64957,4.5


In [ ]:
user_rating_data.sha

In [38]:
sample_data = user_rating_data.sample(10000000)

In [40]:
pt = user_rating_data.pivot_table(values="rating",index = "userId",columns="movie")

C:\Users\gandl\anaconda3\Lib\site-packages\pandas\core\reshape\reshape.py:143: RuntimeWarning: overflow encountered in scalar multiply
  num_cells = num_rows * num_columns


IndexError: index 1196752560 is out of bounds for axis 0 with size 1196742495

In [ ]:
pt

In [30]:
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
simiraity_score = cosine_similarity(pt)
simiraity_score

array([[1.        , 0.        , 0.        , ..., 0.06291708, 0.        ,
        0.01746565],
       [0.        , 1.        , 0.12429498, ..., 0.02413984, 0.17059464,
        0.1131753 ],
       [0.        , 0.12429498, 1.        , ..., 0.08098382, 0.13660585,
        0.17019275],
       ...,
       [0.06291708, 0.02413984, 0.08098382, ..., 1.        , 0.04260878,
        0.08520194],
       [0.        , 0.17059464, 0.13660585, ..., 0.04260878, 1.        ,
        0.22867673],
       [0.01746565, 0.1131753 , 0.17019275, ..., 0.08520194, 0.22867673,
        1.        ]])

In [32]:
simiraity_score_df = pd.DataFrame(simiraity_score)
simiraity_score_df

,0,1,2,3,4,5,6,7,8,9,...,661,662,663,664,665,666,667,668,669,670
0,1.000000,0.000000,0.000000,0.074482,0.016818,0.000000,0.083884,0.000000,0.012843,0.000000,...,0.000000,0.000000,0.014474,0.043719,0.000000,0.000000,0.000000,0.062917,0.000000,0.017466
1,0.000000,1.000000,0.124295,0.118821,0.103646,0.000000,0.212985,0.113190,0.113333,0.043213,...,0.477306,0.063202,0.077745,0.164162,0.466281,0.425462,0.084646,0.024140,0.170595,0.113175
2,0.000000,0.124295,1.000000,0.081640,0.151531,0.060691,0.154714,0.249781,0.134475,0.114672,...,0.161205,0.064198,0.176134,0.158357,0.177098,0.124562,0.124911,0.080984,0.136606,0.170193
3,0.074482,0.118821,0.081640,1.000000,0.130649,0.079648,0.319745,0.191013,0.030417,0.137186,...,0.114319,0.047228,0.136579,0.254030,0.121905,0.088735,0.068483,0.104309,0.054512,0.211609
4,0.016818,0.103646,0.151531,0.130649,1.000000,0.063796,0.095888,0.165712,0.086616,0.032370,...,0.191029,0.021142,0.146173,0.224245,0.139721,0.058252,0.042926,0.038358,0.062642,0.225086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666,0.000000,0.425462,0.124562,0.088735,0.058252,0.000000,0.232051,0.069005,0.066412,0.032653,...,0.342283,0.050754,0.074080,0.124214,0.351207,1.000000,0.091597,0.018416,0.153111,0.127995
667,0.000000,0.084646,0.124911,0.068483,0.042926,0.019563,0.058773,0.112366,0.194493,0.098561,...,0.074089,0.059010,0.093021,0.082525,0.114487,0.091597,1.000000,0.000000,0.178017,0.135387
668,0.062917,0.024140,0.080984,0.104309,0.038358,0.024583,0.073151,0.055143,0.029291,0.060549,...,0.015960,0.025953,0.077927,0.101707,0.028773,0.018416,0.000000,1.000000,0.042609,0.085202
669,0.000000,0.170595,0.136606,0.054512,0.062642,0.019465,0.096240,0.247687,0.384429,0.158650,...,0.183662,0.122126,0.123407,0.143380,0.159479,0.153111,0.178017,0.042609,1.000000,0.228677


In [36]:
import pandas as pd

# Create a pivot table as a sparse DataFrame
pt = pd.pivot_table(
    user_rating_data,
    values="rating",
    index="userId",
    columns="movieId",
    fill_value=0,
    aggfunc='mean'
).astype(pd.SparseDtype("float", 0))


C:\Users\gandl\anaconda3\Lib\site-packages\pandas\core\reshape\reshape.py:143: RuntimeWarning: overflow encountered in scalar multiply
  num_cells = num_rows * num_columns


ValueError: negative dimensions are not allowed

In [61]:
# treshold value of the similarity between users. If treshold > 0.5, similar users
# returns the index of the similar user
def similar_users(userid,simiraity_treshold = 0.5):
    index = userid - 1
    frame = simiraity_score_df[index]
    a = frame[frame >= simiraity_treshold]
    similar_users = a.index.values + 1 #Because index starts from 0 and our userid starts from 1
    return similar_users

# returns the movies list seen by the user
def user_seen_movies(userid):
    movie_list_index =  ratings[ratings["userId"] == userid]["movieId"].values
    return set(movie_list_index)

def seen_movies_by_similar_users(userid):
    similar_user_list = similar_users(userid=userid)
    movie_list_index = []
    for i in similar_user_list:
        movie_list_index.extend(ratings[ratings["userId"] == i]["movieId"].values)
    return set(movie_list_index)


def recommendations_based_upon_user(user_id):
    user_set = user_seen_movies(userid = user_id)
    all_set = seen_movies_by_similar_users(userid=user_id)
    set_difference = all_set.difference(user_set)
    rec_mov = []
    for i in movie_order:
        if (i in set_difference):
            rec_mov.append(i)
    return rec_mov
recommendations_based_upon_user(2)

[318,
 608,
 111,
 293,
 32,
 1,
 246,
 36,
 16,
 475,
 6,
 307,
 58,
 555,
 529,
 348,
 280,
 471,
 595,
 337,
 25,
 175,
 345,
 11,
 171,
 281,
 594,
 230,
 125,
 431,
 247,
 215,
 34,
 40,
 147,
 422,
 353,
 141,
 224,
 596,
 417,
 477,
 440,
 521,
 151,
 531,
 94,
 54,
 257,
 524,
 412,
 21,
 179,
 368,
 648,
 45,
 304,
 240,
 163,
 289,
 380,
 490,
 450,
 89,
 553,
 198,
 249,
 218,
 780,
 270,
 27,
 361,
 640,
 342,
 378,
 376,
 207,
 662,
 195,
 426,
 340,
 203,
 491,
 277,
 610,
 708,
 22,
 433,
 227,
 2,
 421,
 543,
 236,
 799,
 453,
 180,
 333,
 510,
 303,
 282,
 145,
 112,
 516,
 316,
 329,
 122,
 597,
 415,
 288,
 544,
 74,
 237,
 5,
 381,
 104,
 31,
 102,
 489,
 724,
 466,
 276,
 204,
 105,
 252,
 736,
 542,
 540,
 216,
 95,
 4,
 455,
 327,
 520,
 315,
 434,
 231,
 48,
 158,
 44,
 442,
 256,
 196,
 181,
 172,
 420,
 432,
 193,
 344,
 435,
 173,
 355,
 19,
 160]